In [2]:
!pip install -q langchain langchain-openai openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 2.4 MB/s eta 0:00:00


In [3]:
# ====== 匯入模組 ======
from langchain.agents import Tool, initialize_agent
from langchain.agents.agent_types import AgentType
from langchain_openai import ChatOpenAI
from langchain.tools import tool
import os

In [4]:
# ====== 設定 OpenRouter API Key 與 URL ======
os.environ["OPENAI_API_KEY"] = "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"
os.environ["OPENAI_BASE_URL"] = "https://openrouter.ai/api/v1"  # OpenRouter

# ====== 建立 LangChain 的 LLM 客戶端 ======
llm = ChatOpenAI(
    model="deepseek/deepseek-chat-v3-0324:free",
    temperature=0.7,
)

# ====== 自訂工具定義 ======
@tool
def get_weather(location: str) -> str:
    """提供某地區的天氣資訊。"""
    return f"{location} 今天天氣晴，溫度約 25°C。"

@tool
def get_time(location: str) -> str:
    """提供某地區的目前時間。"""
    return f"{location} 現在時間是下午 3:00。"

@tool
def get_exchange_rate(currency: str) -> str:
    """提供指定貨幣對新台幣的匯率資訊。"""
    currency = currency.upper()
    if currency == "USD":
        return "1 USD = 31.25 TWD"
    elif currency == "JPY":
        return "1 JPY = 0.22 TWD"
    else:
        return f"{currency} 匯率資料暫無法提供"

# ====== 工具列表 ======
tools = [get_weather, get_time, get_exchange_rate]

In [7]:
# ====== 初始化 Agent ======
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

# ====== 互動式多輪對話 ======
print("🤖 AI 助理啟動，可輸入 'exit' 離開對話。")

while True:
    user_input = input("🧑 您：")
    if user_input.lower() in ["exit", "quit", "離開"]:
        print("👋 已結束對話。")
        break
    response = agent.run(user_input)
    print("🤖 機器人：" + response)

🤖 AI 助理啟動，可輸入 'exit' 離開對話。
🧑 您：請問台北現在的天氣


> Entering new AgentExecutor chain...
Thought: I need to find the current weather in Taipei.
Action: get_weather
Action Input: "台北"
Observation: 台北 今天天氣晴，溫度約 25°C。
Thought:I now know the final answer.
Final Answer: 台北今天天氣晴，溫度約 25°C。

> Finished chain.
🤖 機器人：台北今天天氣晴，溫度約 25°C。
🧑 您：現在幾點


> Entering new AgentExecutor chain...
Thought: 需要知道使用者所在的地區才能提供準確的時間資訊，但問題中沒有提供具體地點。假設使用者想知道的是台北的時間。

Action: get_time  
Action Input: "台北"  
Observation: 台北 現在時間是下午 3:00。
Thought:Thought: 已經獲得了台北的當前時間資訊，可以回覆使用者。

Final Answer: 台北現在的時間是下午 3:00。

> Finished chain.
🤖 機器人：台北現在的時間是下午 3:00。
🧑 您：請問目前新台幣匯率多少


> Entering new AgentExecutor chain...
Thought: 我需要獲取新台幣的最新匯率資訊，但由於問題中沒有指定要查詢哪種貨幣對新台幣的匯率，我需要先釐清這一點。通常匯率是相對於某種貨幣的，例如美元對新台幣。但問題中沒有明確說明，所以可能需要進一步確認。

不過，考慮到問題問的是「新台幣匯率」，可能是想問新台幣對美元的匯率，這是常見的匯率查詢。因此，我將假設問題是想知道美元對新台幣的匯率。

Action: get_exchange_rate  
Action Input: "USD"  
Observation: 1 USD = 31.25 TWD
Thought:I now know the latest exchange rate of US Dollar to New Taiwa